# 3. Data Preparation & Feature Engineering (TDSP Step 2 — Extended)

Tahap ini merupakan kelanjutan dari *Data Acquisition & Understanding*, yaitu melakukan pembersihan, transformasi, dan pembuatan fitur agar data siap digunakan untuk pemodelan.

**Tujuan utama:**
- Menangani missing values.
- Menghapus kolom yang tidak relevan.
- Menangani data duplikat dan anomali.
- Menyiapkan variabel numerik dan kategorikal untuk modeling.
- Membuat fitur baru jika relevan (misalnya umur mobil).

---

## 3.1 Menangani Missing Values

Langkah pertama adalah memeriksa kolom mana yang memiliki nilai kosong (*missing*).

In [ ]:
# Cek missing values per kolom
df.isna().sum().sort_values(ascending=False)

### Strategi Umum Imputasi

- **Engine_Size:** diisi dengan modus berdasarkan kombinasi `(Type, Year)`.
- **Gear_Type:** diisi dengan modus global.
- Kolom lain: tergantung konteks — misalnya median untuk numerik atau 'Unknown' untuk kategorikal.

Kita buat salinan dataset sebelum melakukan imputasi.

In [ ]:
df_clean = df.copy()

# Imputasi Engine_Size dengan modus per (Type, Year)
engine_group_mode = (
    df_clean.groupby(['Type', 'Year'])['Engine_Size']
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
)

def fill_engine_size(row):
    if pd.notna(row['Engine_Size']):
        return row['Engine_Size']
    key = (row['Type'], row['Year'])
    val = engine_group_mode.get(key, np.nan)
    return val

df_clean['Engine_Size'] = df_clean.apply(fill_engine_size, axis=1)

# Jika masih ada NaN, isi dengan modus global
if df_clean['Engine_Size'].isna().sum() > 0:
    global_mode = df_clean['Engine_Size'].mode().iloc[0]
    df_clean['Engine_Size'].fillna(global_mode, inplace=True)

# Imputasi Gear_Type
if df_clean['Gear_Type'].isna().sum() > 0:
    gear_mode = df_clean['Gear_Type'].mode().iloc[0]
    df_clean['Gear_Type'].fillna(gear_mode, inplace=True)

# Untuk kolom kategorikal lainnya
categorical_cols = df_clean.select_dtypes(include='object').columns
for col in categorical_cols:
    df_clean[col].fillna('Unknown', inplace=True)

print('Missing values setelah imputasi:')
df_clean.isna().sum().sum()

## 3.2 Menghapus Kolom yang Tidak Relevan

Beberapa kolom tidak memiliki informasi prediktif atau hanya bersifat administratif, sehingga bisa dihapus:
- `Link` (URL halaman web)
- `Condition` (jika hanya berisi satu nilai: 'Used')
- `Negotiable` (informasi negosiasi harga, tidak relevan untuk prediksi harga dasar)

In [ ]:
cols_to_drop = ['Link', 'Condition', 'Negotiable']
df_clean.drop(columns=[c for c in cols_to_drop if c in df_clean.columns], inplace=True, errors='ignore')
df_clean.shape

## 3.3 Menghapus Duplikat dan Menangani Outlier Harga

Duplikat data akan dihapus untuk menghindari bias.
Selain itu, harga yang terlalu ekstrem juga perlu difilter agar model tidak overfit pada outlier.

In [ ]:
# Hapus duplikat
print('Jumlah duplikat sebelum dihapus:', df_clean.duplicated().sum())
df_clean = df_clean.drop_duplicates().reset_index(drop=True)

# Pastikan kolom Price numerik
df_clean = df_clean[pd.to_numeric(df_clean['Price'], errors='coerce').notna()]
df_clean['Price'] = df_clean['Price'].astype(float)

# Hapus outlier harga ekstrem (1% bawah dan 99% atas)
q1, q99 = df_clean['Price'].quantile([0.01, 0.99])
df_clean = df_clean[(df_clean['Price'] >= q1) & (df_clean['Price'] <= q99)]

print('Data setelah bersih:', df_clean.shape)

## 3.4 Exploratory Data Analysis Singkat

Visualisasi awal untuk memahami hubungan antar variabel numerik dan target (Price).

In [ ]:
# Kolom numerik utama
numeric_cols = ['Year', 'Engine_Size', 'Mileage', 'Price']

# Histogram
df_clean[numeric_cols].hist(bins=30, figsize=(12, 8))
plt.tight_layout()
plt.show()

# Scatterplot sederhana dengan Price
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
sns.regplot(data=df_clean, x='Year', y='Price', ax=axes[0], scatter_kws={'s':5})
sns.regplot(data=df_clean, x='Engine_Size', y='Price', ax=axes[1], scatter_kws={'s':5})
sns.regplot(data=df_clean, x='Mileage', y='Price', ax=axes[2], scatter_kws={'s':5})
axes[0].set_title('Year vs Price')
axes[1].set_title('Engine_Size vs Price')
axes[2].set_title('Mileage vs Price')
plt.tight_layout()
plt.show()

### Insight Awal
- **Year ↑ → Price ↑:** mobil yang lebih baru cenderung lebih mahal.
- **Mileage ↑ → Price ↓:** mobil yang lebih sering digunakan (jarak tempuh tinggi) cenderung lebih murah.
- **Engine_Size ↑ → Price ↑:** kapasitas mesin lebih besar umumnya menandakan mobil dengan segmen harga lebih tinggi.

## 3.5 Menambahkan Fitur Baru (Opsional)

Kita bisa menambahkan fitur turunan seperti **umur mobil** untuk memperkuat model.

Misalnya: `Car_Age = 2025 - Year`.

In [ ]:
df_clean['Car_Age'] = 2025 - df_clean['Year']
df_clean[['Year', 'Car_Age']].head()

## 3.6 Simpan Dataset Bersih (Opsional)

Langkah ini membantu agar proses modeling berikutnya bisa dimulai langsung dari dataset yang sudah bersih.

In [ ]:
df_clean.to_csv('./Dataset/UsedCarsSA_Clean.csv', index=False)
print('Dataset bersih berhasil disimpan!')